<a href="https://colab.research.google.com/github/Elwing-Chou/tibaml0315/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
from urllib.request import urlretrieve

url = "https://github.com/Elwing-Chou/tibaml0315/raw/main/train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/tibaml0315/raw/main/test.csv"
urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7fb12e0b9af0>)

In [49]:
import pandas as pd
train = pd.read_csv("train.csv", encoding="utf-8")
test = pd.read_csv("test.csv", encoding="utf-8")

In [50]:
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [51]:
# 資料預處理: 1. 先把空值填完
total = pd.concat([train, test], axis=0)
total = total.drop(["PassengerId", "Survived"], axis=1)
total

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
413,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [52]:
def cabin(c):
    if pd.isna(c):
        return c
    else:
        return c[0]
total["Cabin"] = total["Cabin"].apply(cabin)

In [53]:
c = total["Ticket"].value_counts()
def ticket(t):
    if pd.isna(t):
        return t
    else:
        return c[t]
total["Ticket"] = total["Ticket"].apply(ticket)

In [54]:
import re
# 平常
# n = "Braund, Mr. Owen Harris"
# n.split(".")[0].split(",")[-1]
def name(n):
    if pd.isna(n):
        return n
    else:
        pat = r".+,(.+?)\..+"  
        mid = re.match(pat, n).group(1)
        return mid.strip()
total["Name"] = total["Name"].apply(name)

pandas篩選: Series/DataFrame[跟妳資料一樣多的True/False]

In [55]:
s = total.isna().sum()
s[s != 0].sort_values(ascending=False)

Cabin       1014
Age          263
Embarked       2
Fare           1
dtype: int64

缺失值

1. 數值型(無限多種的): Age, Sibsp, Parch, Ticket, Fare

2. 類別型(固定數種的): Pclass, Name, Sex, Cabin, Embarked

補缺失值: 捕最可能的

1. 數值型: 中間值(中位數)

2. 類別型: 最常出現的

In [60]:
med = total.median().drop("Pclass")
total = total.fillna(med)

<ipython-input-60-dc1a23f12b47>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  med = total.median().drop("Pclass")


In [65]:
most = total["Embarked"].value_counts().idxmax()
total["Embarked"] = total["Embarked"].fillna(most)

所有類別型的資料還要再做最後一個處理: One-Hot Encoding

Pclass可以不做: 他是有大小關係的類別, 但我實驗過作比較好

Sex可以不做: 二值

一定要做: 

a. 要確定資料處理好

b. 有時候太過稀少你可以為了表格的美觀而把它丟掉 

c. 可以順手用one-hot來補缺失職


In [72]:
c = total["Name"].value_counts()
reserved = c[c > 10].index
def name2(n):
    if n in reserved:
        return n
    else:
        return None
total["Name"] = total["Name"].apply(name2)

In [ ]:
total = pd.get_dummies(total)
total = pd.get_dummies(total, columns=["Pclass"])
total

1. 缺失 
2. One-Hot 
3. 萃取出更多欄位(回頭在做)

In [ ]:
total["Family"] = total["SibSp"] + total["Parch"]
total

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
total_scale = scaler.fit_transform(total)
total_scale = pd.DataFrame(total_scale)
total_scale

In [84]:
import numpy as np
x = np.array(total_scale.iloc[:len(train)])
y = np.array(train["Survived"])
x_predict = np.array(total_scale.iloc[len(train):])
print(x.shape)
print(y.shape)
print(x_predict.shape)

(891, 26)
(891,)
(418, 26)
